In [2]:
# Import necessary tools
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_hub as hub
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


print("TF version:", tf.__version__)
print("TF Hub version:", hub.__version__)

# Check for GPU availablility
print("GPU", "avaible" if tf.config.list_physical_devices("GPU") else "not avaible")

In [3]:
# Import the data
import pathlib
data_dir = pathlib.Path('../input/pizza-not-pizza/pizza_not_pizza')

In [4]:
# Check image count
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [5]:
pizzas = list(data_dir.glob('pizza/*'))
PIL.Image.open(str(pizzas[0]))

In [6]:
not_pizzas = list(data_dir.glob("not_pizza/*.jpg"))
PIL.Image.open(str(not_pizzas[0]))

In [7]:
len(pizzas), len(not_pizzas)

## Load data using a Keras utility

Let's load these images off disk using the helpful [tf.keras.utils.image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) utility.

## Create a dataset

Define some parameters fot the loader:

In [8]:
batch_size = 32
img_height = 180
img_width = 180

In [9]:
# Create training and validation data
train_ds = tf.keras.utils.image_dataset_from_directory(
data_dir,
validation_split=0.2,
subset="training",
seed=42,
image_size=(img_height, img_width),
batch_size=batch_size)

In [10]:
val_ds = tf.keras.utils.image_dataset_from_directory(
data_dir,
validation_split=0.2,
subset="validation",
seed=42,
image_size=(img_height, img_width),
batch_size=batch_size)

In [11]:
# Fing class names
class_names = train_ds.class_names
print(class_names)

## Visualize the data

Here are the first nine images from the training dataset:

In [12]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Build a basic Keras Model

In [13]:
# Build a model
num_classes = len(class_names)
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(num_classes)
])

## Compile the model

In this notebook Adam is chosen as optimizer and SparseCategoricalCrossentropy as loss function.

In [14]:
model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics="accuracy")
                

## Model summary

In [15]:
# View all the layers of the network
model.summary()

## Train the model

In [16]:
epochs=50
history = model.fit(train_ds,
                   validation_data=val_ds,
                   epochs=epochs)

## Visualize training results

Create plots of the loss and accuracy on the training and validation sets:

In [17]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = "Training Accuracy")
plt.plot(epochs_range, val_acc, label = "Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label ="Training Loss")
plt.plot(epochs_range, val_loss, label = "Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

## Data augmentation

Since model seems to overfit, we try Data augmentation

In [18]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal",
                     input_shape=(img_height,
                                 img_width,
                                 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [19]:
# Visualize data augmentation
plt.figure(figsize=(10,10))
for images, _ in train_ds.take(1):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [20]:
# Build a model with data augmentation and add Dropout
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16,3,padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(num_classes, name="outputs")
])

## Compile and train the new model

In [21]:
model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [22]:
model.summary()

In [25]:
epochs = 50
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

## Visualize new training results

There should be less overfitting due to data augmentation and Dropout -layer

In [27]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

## Predict on new data

In [31]:
pizza_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Nachos-cheese.jpg/800px-Nachos-cheese.jpg"
pizza_path = tf.keras.utils.get_file("Nachos", origin=pizza_url)

img = tf.keras.utils.load_img(
    pizza_path, target_size=(img_height, img_width)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence."
     .format(class_names[np.argmax(score)], 100 * np.max(score)))